<a href="https://colab.research.google.com/github/amitxsethi/Text2Sql-AIML/blob/main/gensql_group6_amit_llama7B_full_spider_official_spider_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

import torch
print(f"\nCUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

Tue Nov 25 04:39:29 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   37C    P0             55W /  400W |       0MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

The `nbstripout` tool can help clean up notebook metadata, which might resolve the rendering issue. You'll need to specify the exact filename of this notebook. After running the command, you might need to refresh your browser page for the changes to take effect.

In [2]:
!git clone https://github.com/jieshi10/gensql.git
%cd gensql
!ls -la
print("✓ Repository cloned!")

Cloning into 'gensql'...
remote: Enumerating objects: 149, done.
remote: Counting objects: 100% (149/149), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 149 (delta 29), reused 148 (delta 28), pack-reused 0 (from 0)
Receiving objects: 100% (149/149), 2.42 MiB | 3.46 MiB/s, done.
Resolving deltas: 100% (29/29), done.
/content/gensql
total 132
drwxr-xr-x 10 root root  4096 Nov 25 04:39 .
drwxr-xr-x  1 root root  4096 Nov 25 04:39 ..
drwxr-xr-x  4 root root  4096 Nov 25 04:39 benchmark
drwxr-xr-x  3 root root  4096 Nov 25 04:39 bird_code
drwxr-xr-x  2 root root  4096 Nov 25 04:39 docker
drwxr-xr-x  8 root root  4096 Nov 25 04:39 .git
-rw-r--r--  1 root root  3139 Nov 25 04:39 .gitignore
-rw-r--r--  1 root root  1064 Nov 25 04:39 LICENSE
-rw-r--r--  1 root root 37953 Nov 25 04:39 main.py
drwxr-xr-x  2 root root  4096 Nov 25 04:39 pretrained
-rw-r--r--  1 root root 16684 Nov 25 04:39 process_sql.py
-rw-r--r--  1 root root  4259 Nov 25 04:39 README.md
-rw-r--r--  1 ro

In [3]:
# Fix dependency issues first
!pip install -q jedi  # Fix ipython warning

# Install packages in the right order to avoid conflicts
print("Installing packages (this takes 2-3 minutes)...")

# Step 1: Core ML packages
!pip install -q torch torchvision torchaudio

# Step 2: NLP packages with compatible versions
!pip install -q 'transformers>=4.41.0,<5.0.0'
!pip install -q sentence-transformers
!pip install -q 'nltk>=3.9'  # Fix textblob conflict
!pip install -q openai

# Step 3: SQL and utility packages
!pip install -q 'sqlglot>=23.4'  # Use newer version
!pip install -q sqlparse
!pip install -q func-timeout
!pip install -q scikit-learn

# Step 4: Install vLLM last (takes longest)
print("\nInstalling vLLM (this takes 1-2 minutes)...")
!pip install -q vllm

print("\n✓ All packages installed successfully!")
print("\nInstalled versions:")
!pip list | grep -E 'torch|transformers|vllm|nltk|sqlglot'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 73.4 MB/s eta 0:00:00
Installing packages (this takes 2-3 minutes)...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done

Installing vLLM (this takes 1-2 minutes)...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 370.3/370.3 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.0/355.0 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.0/183.0 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 105.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 96.3 MB/s eta 0:0

In [4]:
import nltk
import ssl

ssl._create_default_https_context = ssl._create_unverified_context
nltk.download('punkt', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)

print("✓ NLTK data downloaded!")

✓ NLTK data downloaded!


In [5]:
print("Fixing sentence_transformers compatibility...")

# Fix SentenceTransformer.py
with open('sentence_transformers/SentenceTransformer.py', 'r') as f:
    content = f.read()
content = content.replace(
    'from huggingface_hub import HfApi, HfFolder, Repository, hf_hub_url, cached_download',
    'from huggingface_hub import HfApi, HfFolder, Repository, hf_hub_url, hf_hub_download'
)
with open('sentence_transformers/SentenceTransformer.py', 'w') as f:
    f.write(content)

# Fix util.py
with open('sentence_transformers/util.py', 'r') as f:
    content = f.read()

content = content.replace(
    'from huggingface_hub import HfApi, hf_hub_url, cached_download, HfFolder',
    '''from huggingface_hub import HfApi, hf_hub_url, hf_hub_download, HfFolder
try:
    from huggingface_hub import cached_download
except ImportError:
    cached_download = None'''
)

with open('sentence_transformers/util.py', 'w') as f:
    f.write(content)

print("✓ Fixed sentence_transformers!")

Fixing sentence_transformers compatibility...
✓ Fixed sentence_transformers!


In [6]:
from huggingface_hub import snapshot_download

print("Downloading BGE model (~1.3 GB)...")
print("This will take 2-3 minutes...\n")

model_path = snapshot_download(
    repo_id='BAAI/bge-large-en-v1.5',
    local_dir='./pretrained/bge-large-en-v1.5',
    local_dir_use_symlinks=False
)

print(f"\n✓ BGE model downloaded!")
!du -sh pretrained/bge-large-en-v1.5

This will take 2-3 minutes...



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:979: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.34G [00:00<?, ?B/s]


✓ BGE model downloaded!
3.8G	pretrained/bge-large-en-v1.5


In [7]:
import os

os.makedirs('benchmark', exist_ok=True)

print("Downloading Spider dataset (~100 MB)...")
!wget -q --show-progress -O benchmark/spider.zip "https://drive.usercontent.google.com/download?id=1TqleXec_OykOYFREKKtschzY29dUcVAQ&export=download&confirm=t"

print("\nExtracting...")
!cd benchmark && unzip -q spider.zip && rm spider.zip

print("✓ Spider dataset ready!")
!du -sh benchmark/spider
!ls benchmark/spider/

benchmark/spider.zi 100%[===================>]  95.12M  68.1MB/s    in 1.4s    

Extracting...
✓ Spider dataset ready!
878M	benchmark/spider
database      dev.json	  tables.json	  train_others.json
dev_gold.sql  README.txt  train_gold.sql  train_spider.json


In [8]:
from huggingface_hub import snapshot_download
import os

print("="*70)
print("DOWNLOADING CODELLAMA-7B-INSTRUCT")
print("="*70)
print("\nModel size: ~13 GB")
print("Time: 10-15 minutes\n")

model_path = snapshot_download(
    repo_id='codellama/CodeLlama-7b-Instruct-hf',
    local_dir='./models/CodeLlama-7b-Instruct',
    local_dir_use_symlinks=False
)

print(f"\n✓ CodeLlama-7B downloaded!")
print(f"Location: {model_path}")

# Check size
!du -sh ./models/CodeLlama-7b-Instruct

DOWNLOADING CODELLAMA-7B-INSTRUCT

Model size: ~13 GB
Time: 10-15 minutes



Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/646 [00:00<?, ?B/s]

USE_POLICY.md: 0.00B [00:00, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]


✓ CodeLlama-7B downloaded!
Location: /content/gensql/models/CodeLlama-7b-Instruct
26G	./models/CodeLlama-7b-Instruct


In [9]:
!lsof -ti :8000 && echo "⚠️  Port in use" || echo "✓ Port free"

✓ Port free


In [10]:
# Update the LLM client to use port 8000
with open('main.py', 'r') as f:
    code = f.read()

# Change port from 8080 to 8000
code = code.replace("port=8080", "port=8000")
code = code.replace("LLMClient(host='127.0.0.1', port=8080)",
                   "LLMClient(host='127.0.0.1', port=8000)")
code = code.replace("DB_NAME: str='bird'", "DB_NAME: str='spider'")

with open('main.py', 'w') as f:
    f.write(code)

print("✓ Updated main.py to use port 8000")

✓ Updated main.py to use port 8000


In [11]:
import subprocess
import time
import requests

# Kill any old processes
print("Stopping old vLLM processes...")
subprocess.run("pkill -9 -f vllm", shell=True)
time.sleep(3)
print("✓ Stopped\n")

print("="*70)
print("STARTING CODELLAMA-7B ON A100 (40GB)")
print("="*70)
print("\n⚙️ Settings:")
print("   • Model: ./models/CodeLlama-7b-Instruct")
print("   • Max context: 2048 tokens")
print("   • GPU memory: 0.85")
print("   • Max sequences: 16")
print("\n🕐 Loading model (60-90 seconds)...\n")

# Start vLLM with correct path
subprocess.Popen([
    'python', '-m', 'vllm.entrypoints.openai.api_server',
    '--host', '127.0.0.1',
    '--port', '8000',
    '--model', './models/CodeLlama-7b-Instruct',  # ← Correct path
    '--served-model-name', 'CodeLlama-7B-Instruct',
    '--max-model-len', '2048',
    '--gpu-memory-utilization', '0.85',
    '--dtype', 'float16',
    '--max-num-seqs', '16',
    '--disable-log-requests'
], stdout=open('vllm_codellama.log', 'w'), stderr=subprocess.STDOUT)

# Wait for server
print("Waiting for server to start...")
for i in range(15):
    time.sleep(10)
    try:
        r = requests.get('http://127.0.0.1:8000/health', timeout=2)
        if r.status_code == 200:
            models = requests.get('http://127.0.0.1:8000/v1/models').json()
            print(f"\n{'='*70}")
            print(f"✓ SERVER READY after {(i+1)*10} seconds!")
            print(f"✓ Model: {models['data'][0]['id']}")
            print('='*70)
            print("\n🎉 Ready to run Gen-SQL!\n")
            break
    except:
        print(f"⏳ {(i+1)*10}s...")
else:
    print("\n❌ Timeout. Checking logs:\n")
    subprocess.run("tail -30 vllm_codellama.log", shell=True)

Stopping old vLLM processes...
✓ Stopped

STARTING CODELLAMA-7B ON A100 (40GB)

⚙️ Settings:
   • Model: ./models/CodeLlama-7b-Instruct
   • Max context: 2048 tokens
   • GPU memory: 0.85
   • Max sequences: 16

🕐 Loading model (60-90 seconds)...

Waiting for server to start...
⏳ 10s...
⏳ 20s...
⏳ 30s...
⏳ 40s...
⏳ 50s...
⏳ 60s...
⏳ 70s...
⏳ 80s...
⏳ 90s...
⏳ 100s...
⏳ 110s...
⏳ 120s...

✓ SERVER READY after 130 seconds!
✓ Model: CodeLlama-7B-Instruct

🎉 Ready to run Gen-SQL!



In [12]:
!tail -100 vllm_codellama.log

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.60s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:06<00:00,  3.42s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:06<00:00,  3.15s/it]
(EngineCore_DP0 pid=2550) 
(EngineCore_DP0 pid=2550) INFO 11-25 04:46:43 [default_loader.py:314] Loading weights took 6.53 seconds
(EngineCore_DP0 pid=2550) INFO 11-25 04:46:44 [gpu_model_runner.py:3338] Model loading took 12.5563 GiB memory and 34.102175 seconds
(EngineCore_DP0 pid=2550) INFO 11-25 04:46:52 [backends.py:631] Using cache directory: /root/.cache/vllm/torch_compile_cache/319dc32337/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=2550) INFO 11-25 04:46:52 [backends.py:647] Dynamo bytecode transform time: 7.76 s
(EngineCore_DP0 pid=2550) INFO 11-25 04:46:55 [backends.py:251] Cache the graph for dynamic shape for later use
(EngineCor

In [13]:
!grep -n "spider"  main.py | head -20

79:DB_NAME: str='spider'
80:# DB_NAME: str='spider'
94:elif DB_NAME == 'spider':
95:    # spider
96:    DB_SET_PATH: str='benchmark/spider/database/*'
97:    # DB_SET_PATH: str='spider_code/spider_ext/*'
98:    TRAINING_SET_PATHS: List[str]=['benchmark/spider/train_others.json', 'benchmark/spider/train_spider.json']
100:    DEV_SET_PATH: str='benchmark/spider/dev.json'


In [14]:
import nltk
import ssl

# Fix SSL
ssl._create_default_https_context = ssl._create_unverified_context

# Download the new punkt_tab (required for NLTK 3.9+)
print("Downloading punkt_tab...")
nltk.download('punkt_tab', quiet=False)
nltk.download('averaged_perceptron_tagger_eng', quiet=False)

# Also download the old ones as backup
nltk.download('punkt', quiet=False)
nltk.download('averaged_perceptron_tagger', quiet=False)

print("\n✓ All NLTK data downloaded!")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...



✓ All NLTK data downloaded!


[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [15]:
# Create output directory
print("\n2. Creating output directory...")
import os
os.makedirs('output', exist_ok=True)
print("   ✓ Created")


2. Creating output directory...
   ✓ Created


In [16]:
!python main.py

Use Model: CodeLlama-7B-Instruct
2025-11-25 04:54:32.512187: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-25 04:54:32.527686: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764046472.548944    4907 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764046472.559139    4907 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764046472.583665    4907 computation_placer.cc:177] computation placer already registered. Please

In [17]:
import glob
import json

# Find the output file
output_files = glob.glob('output/*.json')
print(f"Output file: {output_files[0]}\n")

# Quick stats
with open(output_files[0], 'r') as f:
    results = json.load(f)

results = [r for r in results if r is not None]
print(f"Total examples processed: {len(results)}")
print(f"Avg time per example: {sum(r['time'] for r in results)/len(results):.2f}s")
print(f"Total time: {sum(r['time'] for r in results):.2f}s\n")

# Show first few questions
print("Questions processed:")
for i, r in enumerate(results[:5]):
    print(f"  {i+1}. {r['question'][:60]}...")

Output file: output/CodeLlama-7B-Instruct-spider-gtrr_column-S30C200-SS4CS6-L5-q_skltn-sql_skltn-sql_post_process-fs8-dev.json

Total examples processed: 778
Avg time per example: 19.36s
Total time: 15062.39s

Questions processed:
  1. How many singers do we have?...
  2. What is the total number of singers?...
  3. Show name, country, age for all singers ordered by age from ...
  4. What are the names, countries, and ages for every singer in ...
  5. What is the average, minimum, and maximum age of all singers...


In [20]:
# Fix the conversion script
conversion_script = '''
import json
from copy import deepcopy
import glob
import sys

# Auto-detect latest output file
output_files = glob.glob('output/*-spider-*.json')
output_files = [f for f in output_files if '_report' not in f and '_errors' not in f and '_gold' not in f and '_pred' not in f and '_failed' not in f]

if not output_files:
    print("❌ No output files found!")
    sys.exit(1)

FILENAME = output_files[0].replace('.json', '')
INPUT_FILE = FILENAME + '.json'
print(f"📂 Processing: {INPUT_FILE}")

OUTPUT_GOLD = FILENAME + '-gold.txt'
OUTPUT_PRED = FILENAME + '-pred.txt'

field = 'infer'

# Load Gen-SQL results
with open(INPUT_FILE, encoding='utf-8') as f:
    data = json.load(f)

# Load ground truth
with open('benchmark/spider/dev.json', encoding='utf-8') as f:
    dev_data = json.load(f)

print(f"Total examples: {len(data)}")
print(f"Valid results: {len([r for r in data if r and isinstance(r, dict)])}")
print(f"Null results: {len([r for r in data if not r])}")

# Write gold and pred files
with open(OUTPUT_GOLD, 'w', encoding='utf-8') as fg, \
        open(OUTPUT_PRED, 'w', encoding='utf-8') as fp:

    for i, (r, dev_r) in enumerate(zip(data, dev_data)):
        # Handle null results
        if r is None or not isinstance(r, dict):
            r = {
                'question': dev_r['question'],
                'output': dev_r['query'],
                'db': dev_r['db_id'],
                'infer': 'SELECT'  # Placeholder for failed generation
            }

        # Verify alignment
        assert r['question'] == dev_r['question'], f"Question mismatch at index {i}"
        assert r['db'] == dev_r['db_id'], f"Database mismatch at index {i}"

        # Get gold and predicted SQL
        gold = dev_r['query']
        pred = r.get('infer', 'SELECT').replace('\\n', ' ').replace('\\t', ' ').strip()
        db = r['db']

        # Write to files
        fg.write(gold + '\\t' + db + '\\n')
        fp.write(pred + '\\t' + db + '\\n')

print(f"\\n✓ Conversion complete!")
print(f"   Gold: {OUTPUT_GOLD}")
print(f"   Pred: {OUTPUT_PRED}")
'''

# Write the fixed script
with open('spider_code/convert_output_ext.py', 'w') as f:
    f.write(conversion_script)

print("✓ Fixed conversion script")

✓ Fixed conversion script


In [21]:
# ====================================================================
# OFFICIAL SPIDER EVALUATION
# ====================================================================

import subprocess

print("="*70)
print("OFFICIAL SPIDER EVALUATION")
print("="*70)

# Step 1: Convert to evaluation format
print("\nStep 1: Converting output to evaluation format...")
result = subprocess.run(['python', 'spider_code/convert_output_ext.py'],
                       capture_output=True, text=True)
print(result.stdout)
if result.returncode != 0:
    print("❌ Conversion failed:")
    print(result.stderr)
else:
    print("✓ Conversion complete")

# Step 2: Run official evaluation
print("\nStep 2: Running official Spider evaluation...")
print("This will take 2-3 minutes...\n")

import glob
output_files = glob.glob('output/*-spider-*.json')
output_files = [f for f in output_files if '_report' not in f and '_errors' not in f]
FILE = output_files[0].replace('output/', '').replace('.json', '')

result = subprocess.run([
    'python', 'benchmark/test-suite-sql-eval/evaluation_2.py',
    '--gold', f'output/{FILE}-gold.txt',
    '--pred', f'output/{FILE}-pred.txt',
    '--etype', 'all',
    '--db', 'benchmark/spider/database',
    '--out_file', f'output/{FILE}-failed.json',
    '--table', 'benchmark/spider/tables.json',
    '--progress_bar_for_each_datapoint'
], capture_output=False, text=True)

print("\n" + "="*70)
print("✓ EVALUATION COMPLETE!")
print("="*70)

# Show result files
print("\nGenerated files:")
!ls -lh output/*-gold.txt output/*-pred.txt output/*-failed.json

OFFICIAL SPIDER EVALUATION

Step 1: Converting output to evaluation format...
📂 Processing: output/CodeLlama-7B-Instruct-spider-gtrr_column-S30C200-SS4CS6-L5-q_skltn-sql_skltn-sql_post_process-fs8-dev.json
Total examples: 1034
Valid results: 778
Null results: 256

✓ Conversion complete!
   Gold: output/CodeLlama-7B-Instruct-spider-gtrr_column-S30C200-SS4CS6-L5-q_skltn-sql_skltn-sql_post_process-fs8-dev-gold.txt
   Pred: output/CodeLlama-7B-Instruct-spider-gtrr_column-S30C200-SS4CS6-L5-q_skltn-sql_skltn-sql_post_process-fs8-dev-pred.txt

✓ Conversion complete

Step 2: Running official Spider evaluation...
This will take 2-3 minutes...


✓ EVALUATION COMPLETE!

Generated files:
-rw-r--r-- 1 root root 189K Nov 25 05:13 output/CodeLlama-7B-Instruct-spider-gtrr_column-S30C200-SS4CS6-L5-q_skltn-sql_skltn-sql_post_process-fs8-dev-failed.json
-rw-r--r-- 1 root root 122K Nov 25 05:11 output/CodeLlama-7B-Instruct-spider-gtrr_column-S30C200-SS4CS6-L5-q_skltn-sql_skltn-sql_post_process-fs8-dev-gol

In [28]:
import json

print("="*70)

# Load gold and pred to see the differences
with open('output/CodeLlama-7B-Instruct-spider-gtrr_column-S30C200-SS4CS6-L5-q_skltn-sql_skltn-sql_post_process-fs8-dev-gold.txt') as f:
    gold_lines = f.readlines()

with open('output/CodeLlama-7B-Instruct-spider-gtrr_column-S30C200-SS4CS6-L5-q_skltn-sql_skltn-sql_post_process-fs8-dev-pred.txt') as f:
    pred_lines = f.readlines()


# Calculate normalized EM
import re

def normalize(sql):
    sql = sql.lower().strip()
    sql = re.sub(r'\s+', ' ', sql)
    sql = re.sub(r'\s*,\s*', ', ', sql)
    sql = re.sub(r'\s*\(\s*', '(', sql)
    sql = re.sub(r'\s*\)\s*', ')', sql)
    return sql.strip()

normalized_matches = 0
for g, p in zip(gold_lines, pred_lines):
    gold_sql = g.split('\t')[0].strip()
    pred_sql = p.split('\t')[0].strip()

    if normalize(gold_sql) == normalize(pred_sql):
        normalized_matches += 1

normalized_em = (normalized_matches / 1034) * 100
print("FINAL RESULTS SUMMARY")
print("="*70)
print(f"Normalized EM:                    {normalized_em:.1f}% (case-insensitive)")
print(f"✅ Execution Accuracy (EX):        {correct_count/1034*100:.1f}% ⭐ MAIN METRIC")
print(f"\nGeneration success rate:          {778/1034*100:.1f}%")
print(f"Null/failed generations:          {256/1034*100:.1f}%")
print("="*70)


FINAL RESULTS SUMMARY
Normalized EM:                    26.1% (case-insensitive)
✅ Execution Accuracy (EX):        59.4% ⭐ MAIN METRIC

Generation success rate:          75.2%
Null/failed generations:          24.8%
